In [1]:
import pprint
from types import SimpleNamespace
from OptiHPLCHandler import EmpowerHandler, EmpowerInstrumentMethod
from typing import List, Optional, Union, Dict

In [2]:
# get api address from .env file
import os
from dotenv import load_dotenv
load_dotenv('../.env_vars')
EMPOWER_API_ADDRESS = os.getenv('EMPOWER_API_ADDRESS')

In [3]:
# Create an instance of the EmpowerHandler class
handler = EmpowerHandler(project="WebAPI_test", address=EMPOWER_API_ADDRESS)

handler.connection.default_get_timeout=300
handler.connection.default_post_timeout=300

# Get the list of methods, select one, and get the method details
with handler:
    method_list = handler.GetMethodList()  # Get the list of instrument methods
    method_name = "20240410_test"
    full_method = handler.GetInstrumentMethod(method_name)

In [4]:
# make sample set table for robustness testing
from generate_basic_robustness_methods import generate_basic_robustness_instrument_methods
# generate methods and return the dict of methods
dict_methods = generate_basic_robustness_instrument_methods(handler, full_method)
dict_methods

{'input_method': {'method_name': '20240410_test', 'run_time': '20.00'},
 'rampup': {'method_name': '20240410_test_ramp', 'run_time': '10'},
 'scaled': {'method_name': '20240410_test_cond_10m', 'run_time': '10.0'},
 'plus_strong': {'method_name': '20240410_test_1_0pct', 'run_time': '20.00'},
 'minus_strong': {'method_name': '20240410_test_m1_0pct', 'run_time': '20.00'},
 'iso_start_1': {'method_name': '20240410_test_iso_30_0m', 'run_time': '50.0'},
 'iso_start_2': {'method_name': '20240410_test_iso_60_0m', 'run_time': '80.0'},
 'plus_temp': {'method_name': '20240410_test_2_5C', 'run_time': '20.00'},
 'minus_temp': {'method_name': '20240410_test_m2_5C', 'run_time': '20.00'}}

In [5]:
def sample_set_namer(method_name: str) -> str:
    return f"{method_name}_robustness"

def get_name_and_run_time(dict_methods_method: dict) -> Union[str, str]:
    return dict_methods_method["method_name"], dict_methods_method["run_time"]

def condition_or_equilibrate_column_sampleset_line(column_position: int, dict_methods_method: dict, function:str, runtime:str=None) -> dict:
    if function not in ["Condition Column", "Equilibrate"]:
        raise ValueError("Function must be 'Condition Column' or 'Equilibrate'")
    
    dict_output = {
        "Function": {"member": function},
        "ColumnPosition": {"member": column_position},
    }
    dict_output["Method"], dict_output["RunTime"] = get_name_and_run_time(dict_methods_method)

    if runtime:
        dict_output["RunTime"] = runtime

    return dict_output

def injection_sampleset_line(column_position: str, vial_position: str, sample_name: str, dict_methods_method: dict, injection_volume: int) -> dict:
    dict_output = {
        "ColumnPosition": {"member": column_position},
        "SamplePos": vial_position,
        "InjectionVolume": injection_volume,
        "SampleName": sample_name,
    }
    dict_output["Method"], dict_output["RunTime"] = get_name_and_run_time(dict_methods_method)

    if len(dict_output["SampleName"]) > 30:
        raise ValueError("Sample name too long")

    return dict_output

def post_and_run_experiment(handler: EmpowerHandler, 
                            sample_set_method_name: str, 
                            sample_list: List[Dict[str, str]], 
                            plates: Dict[str, str], 
                            system: str, 
                            node: str,
                            post: bool = False,
                            run: bool = False,
                            ) -> None:
    
    print("Logging in to Empower...")
    with handler:
        print("Log in successful.")

        if post:
            print(f"Posting SampleSet Method {sample_set_method_name}...")
            handler.PostExperiment(
                sample_set_method_name=sample_set_method_name,
                sample_list=sample_list,
                plates=plates,
                audit_trail_message=None,
            )
            print(f"Sample Set Method {sample_set_method_name} posted.")
        
        else:
            print(f"Sample Set Method {sample_set_method_name} not posted.")

        if run:
            print(f"Running SampleSet Method {sample_set_method_name}...")
            handler.RunExperiment(
                sample_set_method=sample_set_method_name,
                sample_set_name=sample_set_method_name,
                system=system,
                node=node,
            )
            print(f"Sample Set Method {sample_set_method_name} running/queued.")

        else:
            print(f"Sample Set Method {sample_set_method_name} not run.")

def generate_basic_robustness_sampleset(
        dict_methods: dict,
        column_position: str = "Position 1", 
        blank_vial_position: str = "1:A,1",
        sample_vial_position: str = "1:A,2",
        sample_name: str = "Sample", 
        injection_volume: int = 3,
        linearity_volumes: list = [0.7, 1, 3, 5, 7, 10],
        equilibration_time: str = "10.0",
        repeat_injections_number: int = 5,
                                        ) -> List[dict]:
    
    # Initialisation
    sample_list = []

    # Startup
    sample_list.append(condition_or_equilibrate_column_sampleset_line(column_position, dict_methods["rampup"], "Condition Column"))
    sample_list.append(condition_or_equilibrate_column_sampleset_line(column_position, dict_methods["scaled"], "Condition Column"))
    sample_list.append(condition_or_equilibrate_column_sampleset_line(column_position, dict_methods["input_method"], "Equilibrate", equilibration_time))

    # Carryover
    sample_list.append(injection_sampleset_line(column_position, blank_vial_position, "Pre carryover blank", dict_methods["input_method"], injection_volume))
    sample_list.append(injection_sampleset_line(column_position, sample_vial_position, sample_name, dict_methods["input_method"], injection_volume))
    sample_list.append(injection_sampleset_line(column_position, blank_vial_position, "Post carryover blank", dict_methods["input_method"], injection_volume))

    # Repeat injections
    for i in range(repeat_injections_number):
        sample_list.append(injection_sampleset_line(column_position, sample_vial_position, f"{sample_name}_rep{i+1}", dict_methods["input_method"], injection_volume))

    sample_list.append(injection_sampleset_line(column_position, blank_vial_position, "Post repeatinj blank", dict_methods["input_method"], injection_volume))
  
    # Linearity
    for volume in linearity_volumes:
        sample_list.append(injection_sampleset_line(column_position, sample_vial_position, f"{sample_name}_{volume}uL", dict_methods["input_method"], volume))

    # +/- strong eluent
    dict_plus_strong_equilibrate = condition_or_equilibrate_column_sampleset_line(column_position, dict_methods["plus_strong"], "Equilibrate", equilibration_time)
    sample_list.append(dict_plus_strong_equilibrate)
    dict_plus_strong_injection = injection_sampleset_line(column_position, sample_vial_position, f"{sample_name}_p_str", dict_methods["plus_strong"], injection_volume)
    sample_list.append(dict_plus_strong_injection)

    dict_minus_strong_equilibrate = condition_or_equilibrate_column_sampleset_line(column_position, dict_methods["minus_strong"], "Equilibrate", equilibration_time)
    sample_list.append(dict_minus_strong_equilibrate)
    dict_minus_strong_injection = injection_sampleset_line(column_position, sample_vial_position, f"{sample_name}_m_str", dict_methods["minus_strong"], injection_volume)
    sample_list.append(dict_minus_strong_injection)

    # Isocratic start
    dict_iso_start_1_equilibrate = condition_or_equilibrate_column_sampleset_line(column_position, dict_methods["iso_start_1"], "Equilibrate", equilibration_time)
    sample_list.append(dict_iso_start_1_equilibrate)
    dict_iso_start_1_injection = injection_sampleset_line(column_position, sample_vial_position, f"{sample_name}_iso1", dict_methods["iso_start_1"], injection_volume)
    sample_list.append(dict_iso_start_1_injection)

    dict_iso_start_2_injection = injection_sampleset_line(column_position, sample_vial_position, f"{sample_name}_iso2", dict_methods["iso_start_2"], injection_volume)
    sample_list.append(dict_iso_start_2_injection)

    # +/- temperature
    dict_plus_temp_equilibrate = condition_or_equilibrate_column_sampleset_line(column_position, dict_methods["plus_temp"], "Equilibrate", equilibration_time)
    sample_list.append(dict_plus_temp_equilibrate)
    dict_plus_temp_injection = injection_sampleset_line(column_position, sample_vial_position, f"{sample_name}_p_temp", dict_methods["plus_temp"], injection_volume)
    sample_list.append(dict_plus_temp_injection)

    dict_minus_temp_equilibrate = condition_or_equilibrate_column_sampleset_line(column_position, dict_methods["minus_temp"], "Equilibrate", equilibration_time)
    sample_list.append(dict_minus_temp_equilibrate)
    dict_minus_temp_injection = injection_sampleset_line(column_position, sample_vial_position, f"{sample_name}_m_temp", dict_methods["minus_temp"], injection_volume)
    sample_list.append(dict_minus_temp_injection)

    return sample_list

plate = "ANSI-48Vial2mLHolder"
plates = {"1": plate, "2": plate}
system = "12_5208_HCLASS_BIN"
node = "Epdkhqr01048"
project = "WebAPI_test"
method_name = "20240410_test"

sample_list = generate_basic_robustness_sampleset(dict_methods)
sample_set_method_name = sample_set_namer(method_name)

# post and run experiment
post_and_run_experiment(
        handler=handler,
        sample_set_method_name=sample_set_method_name,
        sample_list=sample_list,
        plates=plates,
        system=system,
        node=node,
        post=True,
        run=False
    )


Logging in to Empower...
Log in successful.
Posting SampleSet Method 20240410_test_robustness...
Sample Set Method 20240410_test_robustness posted.
Sample Set Method 20240410_test_robustness not run.


In [39]:
dict_methods={
    "input_method": {
        "method_name": "test",
        "run_time": "20.0",            
                     },
    "rampup": {
        "method_name": "test_rampup",
        "run_time": "10.0",            
                     },
    "plus_strong": {
        "method_name": "test_plus_strong",
        "run_time": "20.0",            
                     },
    "minus_strong": {
        "method_name": "test_minus_strong",
        "run_time": "20.0",            
                     },
    "iso_start_1": {
        "method_name": "test_iso_start_1",
        "run_time": "50.0",            
                     },
    "iso_start_2": {
        "method_name": "test_iso_start_2",
        "run_time": "80.0",            
                     },
    "plus_temp": {
        "method_name": "test_plus_temp",
        "run_time": "20.0",            
                     },
    "minus_temp": {
        "method_name": "test_minus_temp",
        "run_time": "20.0",            
                     },
}

In [9]:
# add wash and re-equilibration
from types import SimpleNamespace
from empower_implementation.empower_tools import determine_strong_eluent
method=SimpleNamespace(
gradient_table = [
    {"Time": 0, "CompositionA": 95, "CompositionB": 5, "CompositionC": 0, "CompositionD": 0},
    {"Time": 10, "CompositionA": 50, "CompositionB": 50, "CompositionC": 0, "CompositionD": 0},
],
method_name = "test",
)

wash_strong_composition = 95
wash_time = 2
re_equilibration_time = 7

strong_eluent, list_weak_eluents = determine_strong_eluent(method.gradient_table)

gradient_portion_end_time = method.gradient_table[-1]["Time"]
gradient_start_composition_strong = method.gradient_table[0][strong_eluent]



wash_rows = [{
    "Time": gradient_portion_end_time + 0.1,
    strong_eluent: wash_strong_composition,
},
{
    "Time": gradient_portion_end_time + wash_time + 0.1,
    strong_eluent: wash_strong_composition,
},
{
    "Time": gradient_portion_end_time + wash_time + 0.2,
    strong_eluent: gradient_start_composition_strong,
},
{
    "Time": gradient_portion_end_time + wash_time + re_equilibration_time,
    strong_eluent: gradient_start_composition_strong,
}]

method.gradient_table += wash_rows

method.gradient_table






[{'Time': 0,
  'CompositionA': 95,
  'CompositionB': 5,
  'CompositionC': 0,
  'CompositionD': 0},
 {'Time': 10,
  'CompositionA': 50,
  'CompositionB': 50,
  'CompositionC': 0,
  'CompositionD': 0},
 {'Time': 10.1, 'CompositionB': 95},
 {'Time': 12.1, 'CompositionB': 95},
 {'Time': 12.2, 'CompositionB': 5},
 {'Time': 19, 'CompositionB': 5}]